In [1]:
%load_ext autoreload
%autoreload 2

### Content / Style metric

In [2]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.models import Model
from skimage import io
import numpy as np

import keras.backend as K

from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


Define the model:

In [3]:
base_model = MobileNetV2(alpha=1.0, weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

Content layer:

In [4]:
content_layer = 'block_7_depthwise_relu'

In [5]:
model_content = Model(inputs=base_model.input, outputs=base_model.get_layer(content_layer).output)

Style layers:

In [6]:
style_layers = ['block_3_depthwise_relu', 'block_5_depthwise_relu',
                'block_7_depthwise_relu']

In [7]:
models_style = []
for layer in style_layers:
    models_style.append(Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output))

Load the image:

In [4]:
img_nrows = img_ncols = 224

In [10]:
def preprocess(urls):
    from keras.preprocessing import image
    imgs = map(io.imread, urls)
    imgs = map(image.array_to_img, imgs)
    imgs = [img.resize((img_nrows, img_ncols)) for img in imgs]
    imgs = map(image.img_to_array, imgs)
    imgs = map(preprocess_input, imgs)
    return np.array(list(imgs))

Content and style losses implementation:

In [10]:
def gram_matrix(x):
    features = x.reshape(x.shape[1] * x.shape[2], -1)
    gram = features @ features.T
    return gram


def style_loss(x, y):
    S = gram_matrix(x)
    C = gram_matrix(y)
    channels = 3
    size = img_nrows * img_ncols
    return ((S - C) ** 2).sum() / (4.0 * (channels ** 2) * (size ** 2))


def content_loss(x, y):
    return ((x - y) ** 2).sum()

Test with different images:

In [10]:
vangog1 = "http://artchallenge.me/painters/21/532.jpg"
vangog2 = "https://studio57.ru/wp-content/uploads/2016/12/vystavkavangoga-orel-6.jpg"

In [11]:
json_user_images_links = {
        "nm.tismoney": ["https://scontent-arn2-1.cdninstagram.com/vp/d823d5d55dee8c31dff13928b51fc99f/5C965EEF/t51.2885-19/s320x320/41105233_1812609548786318_770863607914168320_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/a05b4281cbd81433c49e620fea1ecc94/5C9B1DF8/t51.2885-15/sh0.08/e35/s640x640/43556265_948345168689036_2367865772941770752_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/fddc640533b5b38997a200e33e2debfd/5CA2FCF3/t51.2885-15/sh0.08/e35/s640x640/41449619_1794170000700220_4264116412768845824_n.jpg"],
        "nastiatolstun": ["https://scontent-arn2-1.cdninstagram.com/vp/89030f6caf72ee651eba481cd6b202e2/5CA12CE2/t51.2885-19/s320x320/40482427_2078359475559046_3340041571830595584_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/2dd1c5d6ad62b671d24623d039fd7046/5CAE345D/t51.2885-15/sh0.08/e35/p640x640/44600295_186845698920576_8509778804713339012_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/aec826cf6be643f342b30eda8ca0b1d7/5C999C7D/t51.2885-15/sh0.08/e35/p640x640/43914258_256833778327187_4891278425613460787_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/33af47676784264604c79fe56a5b05fa/5C95C64D/t51.2885-15/sh0.08/e35/s640x640/45754618_2233136490304606_2132765987628434534_n.jpg"],
        "ushakov.roma": ["https://scontent-arn2-1.cdninstagram.com/vp/71a1b4a63ffa01561f5f6e20b7bc3254/5C9A5D32/t51.2885-19/s150x150/14693933_671218896371000_288462776832098304_a.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/2ef3dd7c048004a55653e30164daae8e/5C8D39E4/t51.2885-15/sh0.08/e35/s640x640/43985247_138677503772423_1881834236909653291_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/e9eafce4256e58c1e5d0fd5a6390da91/5C947C40/t51.2885-15/sh0.08/e35/s640x640/42068933_331899987569953_6725643588537206289_n.jpg", "https://scontent-arn2-1.cdninstagram.com/vp/80bc707ea70f1d43ee1231d4dd347592/5C9A8B90/t51.2885-15/sh0.08/e35/s640x640/42003417_283527345593631_3434722517032611667_n.jpg"]
    }

In [15]:
# img1 = preprocess([vangog1])
# img2 = preprocess([vangog2])
imgs = preprocess(json_user_images_links["nm.tismoney"])
img1 = imgs[0]
img2 = imgs[1]

In [17]:
%%time
img1_content = model_content.predict(img1[np.newaxis,:])
img2_content = model_content.predict(img2[np.newaxis,:])
C_loss = content_loss(img1_content, img2_content)
print('Content loss:', C_loss)

Content loss: 33590.348
CPU times: user 776 ms, sys: 212 ms, total: 988 ms
Wall time: 987 ms


In [24]:
%%time
S_loss = 0
for style_model in models_style:
    img1_style = style_model.predict(img1[np.newaxis,:])
    img2_style = style_model.predict(img2[np.newaxis,:])
    S_loss += style_loss(img1_style, img2_style)
print('Style loss:', S_loss)

Style loss: 0.020542184175304828
CPU times: user 2.01 s, sys: 460 ms, total: 2.47 s
Wall time: 1.91 s


---

In [54]:
def metric_emb(self, img1, img2):
    img1_emb = self.base_model.predict(img1)
    img1_emb = img1_emb.sum(axis=1).sum(axis=1).mean(axis=0).reshape(1, -1)
    img2_emb = self.base_model.predict(img2)
    img2_emb = img2_emb.sum(axis=1).sum(axis=1).mean(axis=0).reshape(1, -1)
    return float(1 - cosine_similarity(img1_emb, img2_emb))

In [55]:
def metric_content(self, img1, img2):
    img1_content = self.model_content.predsict(img1)
    img2_content = self.model_content.predict(img2)
    C_loss = content_loss(img1_content, img2_content)
    return C_loss

In [56]:
def metric_style(self, img1, img2):
    S_loss = 0
    for style_model in self.models_style:
        img1_style = style_model.predict(img1)
        img2_style = style_model.predict(img2)
        S_loss += style_loss(img1_style, img2_style)
    return S_loss

---

In [1]:
import json

In [2]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.models import Model
from skimage import io
import numpy as np

import keras.backend as K

from sklearn.metrics.pairwise import cosine_similarity

from model import RankModel

Using TensorFlow backend.


In [3]:
rm = RankModel()

In [6]:
json_path = '../instagram_parser/dumps/k_artemkaa.json'
json_user_images_links = ''

with open(json_path) as json_data:
    json_user_images_links = json.load(json_data)
    print(json_user_images_links)

{'leonovali': ['https://scontent-iad3-1.cdninstagram.com/vp/369cd3a87d5095da38b79e69a72dd084/5CB0FFF1/t51.2885-19/s150x150/43984532_185612079014996_3600515210730799104_n.jpg', 'https://scontent-iad3-1.cdninstagram.com/vp/7c92774957221cedcd811ffef771cd5e/5CA778E5/t51.2885-15/e35/42798632_493275527858805_1448326729351550199_n.jpg?se=7&ig_cache_key=MTkwODkxNTU0NDI5NzY3MjI4Nw%3D%3D.2', 'https://scontent-iad3-1.cdninstagram.com/vp/36370180b743b7274e5a16a820ab50b3/5CA4CEF7/t51.2885-15/e35/43817940_197280531163905_5836825885583472763_n.jpg?se=7&ig_cache_key=MTkwNTE5MzA0MTU4MjYyOTUxMg%3D%3D.2', 'https://scontent-iad3-1.cdninstagram.com/vp/0ed5972caf25791f0984bc6055615bec/5CAB87D6/t51.2885-15/e35/44542375_488284875018479_5054560189415800584_n.jpg?se=7&ig_cache_key=MTg5ODA5NTY3NTQyNzk1MjI4MA%3D%3D.2', 'https://scontent-iad3-1.cdninstagram.com/vp/42477f53502539f9e42cacee4e7100ec/5C9F1F58/t51.2885-15/e35/42669021_1440318576071113_2819457406076763417_n.jpg?se=7&ig_cache_key=MTg4NzkxMTA0MDcyNTkyMzEz

In [7]:
rm.update_urls([x for x in json_user_images_links.values()])

In [8]:
rm.list_urls

[['https://scontent-iad3-1.cdninstagram.com/vp/369cd3a87d5095da38b79e69a72dd084/5CB0FFF1/t51.2885-19/s150x150/43984532_185612079014996_3600515210730799104_n.jpg',
  'https://scontent-iad3-1.cdninstagram.com/vp/7c92774957221cedcd811ffef771cd5e/5CA778E5/t51.2885-15/e35/42798632_493275527858805_1448326729351550199_n.jpg?se=7&ig_cache_key=MTkwODkxNTU0NDI5NzY3MjI4Nw%3D%3D.2',
  'https://scontent-iad3-1.cdninstagram.com/vp/36370180b743b7274e5a16a820ab50b3/5CA4CEF7/t51.2885-15/e35/43817940_197280531163905_5836825885583472763_n.jpg?se=7&ig_cache_key=MTkwNTE5MzA0MTU4MjYyOTUxMg%3D%3D.2',
  'https://scontent-iad3-1.cdninstagram.com/vp/0ed5972caf25791f0984bc6055615bec/5CAB87D6/t51.2885-15/e35/44542375_488284875018479_5054560189415800584_n.jpg?se=7&ig_cache_key=MTg5ODA5NTY3NTQyNzk1MjI4MA%3D%3D.2',
  'https://scontent-iad3-1.cdninstagram.com/vp/42477f53502539f9e42cacee4e7100ec/5C9F1F58/t51.2885-15/e35/42669021_1440318576071113_2819457406076763417_n.jpg?se=7&ig_cache_key=MTg4NzkxMTA0MDcyNTkyMzEzOQ%3D

In [21]:
img_nrows = img_ncols = 224

In [22]:
def preprocess(urls):
    from keras.preprocessing import image
    imgs = map(io.imread, urls)
    imgs = map(image.array_to_img, imgs)
    imgs = [img.resize((img_nrows, img_ncols)) for img in imgs]
    imgs = map(image.img_to_array, imgs)
    imgs = map(preprocess_input, imgs)
    return np.array(list(imgs))

In [23]:
black_url = "https://images.golos.io/DQmPRC8w74CW9zuJfefitE6LHFWfRuuK2P5GWHSiaDHCPgR/pure-black-full-hd-background_1_1280x720.jpg"
img_black = preprocess([black_url])

In [ ]:
# filestr = request.files['image'].read()
nparr = np.fromstring(filestr, np.uint8)
img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
img = model.preprocess_image(img)
print(img.shape)

In [24]:
# BLACK!!
rm.recs(img_black)

image mean: -0.65250736
[0.22046519219875335, 0.22102105617523193, 0.03657316267490387, 0.22314285039901732, 0.03484412431716919, 0.038008767366409305, 0.22694941163063048, 0.03749457001686096, 0.2157383680343628, 0.2218792587518692, 0.22756554186344147, 0.23470010459423066, 0.23493830859661102, 0.2126156508922577, 0.23311271965503694, 0.22257776260375978, 0.03757806122303009, 0.2274282306432724, 0.22108855247497558, 0.22898291647434235]


([4, 2, 7, 16, 5, 13, 8, 0, 1, 18, 9, 15, 3, 6, 17, 10, 19, 14, 11, 12],
 [0.9659950168754989,
  0.9576477356010681,
  0.953199455886951,
  0.9527963852559741,
  0.9507170645113953,
  0.10776713068135997,
  0.09269158408690065,
  0.06987188742809461,
  0.06718834201035694,
  0.06686249001498568,
  0.06304519604393766,
  0.059673027869114235,
  0.056944952606547805,
  0.038568010795813054,
  0.03625641547452595,
  0.03559351787496651,
  0.028750856174895496,
  0.008813398684096039,
  0.00114997784732183,
  0.0])

In [25]:
emily_url = "https://wallpapertag.com/wallpaper/full/e/c/5/617235-emilia-clarke-wallpapers-1920x1200-for-xiaomi.jpg"
img_emily = preprocess([emily_url])

In [26]:
# EMILY!!
rm.recs(img_emily)

image mean: 0.28993356
[0.16525322794914246, 0.18003207743167876, 0.02381056547164917, 0.17032151222229003, 0.030515146255493165, 0.030740922689437865, 0.16323318779468537, 0.02297348976135254, 0.19637847542762757, 0.1771709442138672, 0.20006031692028045, 0.19784148633480073, 0.21208738088607787, 0.16768980622291565, 0.21473135948181152, 0.1871421903371811, 0.028137776255607604, 0.19211968779563904, 0.16232483088970184, 0.13834345638751983]


([7, 2, 16, 4, 5, 19, 18, 6, 0, 13, 3, 9, 1, 15, 17, 8, 11, 10, 12, 14],
 [0.9712517792182414,
  0.96701199873804,
  0.9450947174337635,
  0.9330533610836179,
  0.9319098056014153,
  0.3869039998162105,
  0.26543856693829193,
  0.2608377478875964,
  0.25060626379273937,
  0.2382650181006773,
  0.22493545234153559,
  0.1902431447029923,
  0.1757515322730529,
  0.13973887829447681,
  0.11452790119283802,
  0.09295718249998174,
  0.08554704622406703,
  0.07430869049414215,
  0.013391726342521355,
  0.0])

In [22]:
!nvidia-smi

Sun Dec  2 06:57:09 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   56C    P0    56W / 149W |  10955MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  